In [ ]:
# Case 1
env = OSWorld()
proc = OSWorldProcessor()

task = next(proc)
env.reset(task)

# Pros
# - one class that handles data ops
# - another for env
# - wanna use benchmark tasks? just use the processor
# - wanna process your custom slice of the benchmarks? just use the processor 
# - wanna use your custom task config? just dont use the processor

# Cons
# - additional class 

In [ ]:
# Case 2
env = OSWorld(OSWorldProcessor("evaluation_examples/"), secrets, use_benchmark=True)

env.reset()

# Pros
# - single class object to handle benchmark tasks or custom tasks

# Cons
# - intermingling between data ops and env ops
# - confuses the init for env; maybe a bit too overloading

In [ ]:
# Case 3

# OSWorld <- OSWorldAdapter <- OSWorldProcessor 
# OSWorldEnv -> OSWorld <- OSWorldProcessor

env = OSWorld()
proc = OSWorldProcessor("evaluation_examples/")

# Pros
# - scalable, modular, robust

# Cons
# - more effort
# - more maintenance
# - more complexity

In [ ]:
{
  "id": "0c825995-5b70-4526-b663-113f4c999dd2",
  "snapshot": "libreoffice_calc",
  "instruction": "I'm working on a comprehensive report for our environmental policy review meeting next week. I need to integrate key insights from an important document, which is a guidebook on the Green Economy, where I'm particularly interested in the 'Introduction' section. Could you extract this section and compile them into a new Google Doc named 'environment_policy_report (draft)' under /environment_policy folder? This will significantly aid in our discussion on aligning our environmental policies with sustainable and green economic practices. Thanks!",
  "source": "authors",
  "config": [
    {
      "type": "googledrive",
      "parameters": {
        "settings_file": "evaluation_examples/settings/googledrive/settings.yml",
        "operation": [
          "delete"
        ],
        "args": [
          {
            "query": "title = 'environment_policy_report (draft).doc' or title = 'environment_policy_report (draft).docx' or title = 'environment_policy_report (draft)'",
            "trash": false
          }
        ]
      }
    },
    {
      "type": "launch",
      "parameters": {
        "command": [
          "google-chrome",
          "--remote-debugging-port=1337"
        ]
      }
    },
    {
      "type": "launch",
      "parameters": {
        "command": [
          "socat",
          "tcp-listen:9222,fork",
          "tcp:localhost:1337"
        ]
      }
    },
    {
      "type": "login",
      "parameters": {
        "settings_file": "evaluation_examples/settings/google/settings.json",
        "platform": "googledrive"
      }
    },
    {
      "type": "command",
      "parameters": {
        "command": [
          "mkdir",
          "-p",
          "/home/user/Desktop/wwf"
        ]
      }
    },
    {
      "type": "download",
      "parameters": {
        "files": [
          {
            "path": "/home/user/Desktop/wwf/lpr_living_planet_report_2016.pdf",
            "url": "https://drive.google.com/uc?id=19NCdw_MVP6nH5nC6okYYe8U1mJABfTRK&export=download"
          },
          {
            "path": "/home/user/Desktop/wwf/279c656a32_ENGLISH_FULL.pdf",
            "url": "https://drive.google.com/uc?id=1ckH1NetfImQ9EyONTO-ZFWA8m8VIUFvD&export=download"
          },
          {
            "path": "/home/user/Desktop/wwf/7g37j96psg_WWF_AR2021_spreads.pdf",
            "url": "https://drive.google.com/uc?id=1cxLTzmqDKMomOyvho29lvFvhRnb0Y8__&export=download"
          },
          {
            "path": "/home/user/Desktop/GE Guidebook.pdf",
            "url": "https://drive.google.com/uc?id=1KzC_R3eI3Rmgwz5bkcI8Ohv7ebOrU-Is&export=download"
          },
          {
            "path": "/home/user/Desktop/assessing_and_reporting_water_quality(q&a).pdf",
            "url": "https://drive.google.com/uc?id=1LFojf3Weflv3fVdrZrgTY1iUaRdbT9kG&export=download"
          }
        ]
      }
    }
  ],
  "trajectory": "trajectories/0c825995-5b70-4526-b663-113f4c999dd2",
  "related_apps": [
    "libreoffice_calc",
    "chrome",
    "os"
  ],
  "evaluator": {
    "func": "compare_docx_files",
    "result": {
      "type": "googledrive_file",
      "settings_file": "evaluation_examples/settings/googledrive/settings.yml",
      "path": ["environment_policy", "environment_policy_report (draft)"],
      "dest": "environment_policy_report (draft).docx"
    },
    "expected": {
      "type": "cloud_file",
      "path": "https://drive.google.com/uc?id=1A2ti9JncAfIa6ks7FTJWHtYlZo-68FtM&export=download",
      "dest": "environment_policy_report (draft)_gold.docx"
    },
    "options": {
      "content_only": true
    }
  }
}


In [ ]:
env = DesktopEnv()
env.reset()

Googledrive
- settings.yml
  - client_secrets.json
  - credentials.json (write path)

Google
- settings.json

In [2]:
google_credentials_config = {
    "googledrive_settings": {  # settings.yml
        'client_config_backend': 'file',
        'client_config_file': "evaluation_examples/settings/googledrive/client_secrets.json",  # User created.
        'save_credentials': True,
        'save_credentials_backend': 'file',
        'save_credentials_file': "evaluation_examples/settings/googledrive/credentials.json",  # User specified.
        'get_refresh_token': True
    },
    "google_settings": {  # settings.json
        "email": "vincentsimulartest@gmail.com",
        "password": ""
    }
}

# settings.yml (googledrive)
    # client_secrets.json
    # credentials.json path specified by user
# settings.json (google)

In [3]:
google_credentials_config['googledrive_settings']

{'client_config_backend': 'file',
 'client_config_file': 'evaluation_examples/settings/googledrive/client_secrets.json',
 'save_credentials': True,
 'save_credentials_backend': 'file',
 'save_credentials_file': 'evaluation_examples/settings/googledrive/credentials.json',
 'get_refresh_token': True}

In [4]:
google_credentials_config['google_settings']

{'email': 'vincentsimulartest@gmail.com', 'password': ''}